In [0]:
import pandas as pd
import numpy as np
import random
import pickle
import gc
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

## Load in the pre-pitcher pre-processed df

In [0]:
#load in pickled pre_processed df

df1 = pd.read_pickle("/content/RC-v1.1--Data-Cleaning-and-Feature-Engineering/data/pre_pitcher_pre_processed_df_part1.pkl", compression='zip')
df2 = pd.read_pickle("/content/RC-v1.1--Data-Cleaning-and-Feature-Engineering/data/pre_pitcher_pre_processed_df_part2.pkl", compression='zip')

df = pd.concat([df1, df2])

In [184]:
#clear some memory:
del df1
del df2
gc.collect()

154

## Choose the pitcher to model

In [0]:
#df.groupby('pitcher').count().sort_values('pitch_type', ascending=False).head(10)

In [288]:
#dictionary mapping top 10 pitchers with most pitches name with pitcher id

pitcher_id_dict = {'Justin Verlander': 434378.0, 'Trevor Bauer': 545333.0, 
                   'Aaron Nola': 605400.0, 'Lance Lynn': 458681.0, 'Gerrit Cole':
                   543037.0, 'Jacob deGrom':594798.0, 'Zack Greinke':425844.0,
                   'Patrick Corbin': 571578.0, 'Max Scherzer': 453286.0,
                   'Kyle Gibson': 502043.0}

#make pitcher_df subset where pitcher == picther_id
pitcher_id = pitcher_id_dict['Justin Verlander']
# pitcher_id = pitcher_id_dict['Trevor Bauer']
# pitcher_id = pitcher_id_dict['Aaron Nola']
# pitcher_id = pitcher_id_dict['Lance Lynn']
# pitcher_id = pitcher_id_dict['Gerrit Cole']
# pitcher_id = pitcher_id_dict['Jacob deGrom']
# pitcher_id = pitcher_id_dict['Zack Greinke']
# pitcher_id = pitcher_id_dict['Patrick Corbin']
# pitcher_id = pitcher_id_dict['Max Scherzer']
# pitcher_id = pitcher_id_dict['Kyle Gibson']

pitcher_df = df[df['pitcher'] == pitcher_id]
pitcher_df.head()

,pitch_type,game_date,index,release_speed,release_pos_x,release_pos_z,batter,pitcher,zone,hit_location,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,fielder_2,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,stand,p_throws,home_team,away_team,type,bb_type,inning_topbot,sv_id,if_fielding_alignment,score_diff,on_1b_id,ball_high,ball_low,ball_left,ball_right,bases_loaded,batter_swung,in_strikezone,chased,fastball_perc_faced,fastball_chase_perc,fastball_bip_swung_perc,fastball_taken_strike_perc,fastball_est_woba,fastball_babip,fastball_iso_value,breaking_perc_faced,breaking_chase_perc,breaking_bip_swung_perc,breaking_taken_strike_perc,breaking_est_woba,breaking_babip,breaking_iso_value,offspeed_perc_faced,offspeed_chase_perc,offspeed_bip_swung_perc,offspeed_taken_strike_perc,offspeed_est_woba,offspeed_babip,offspeed_iso_value,pitchout_perc_faced,balls,strikes,player_name,events,description,des,pitch_name,of_fielding_alignment,_count,count_cat,pitch_cat
10018,FF,2018-03-31,15196,92.300003,-1.9879,6.6027,448801.0,502043.0,14.0,NaN,2018.0,-0.5666,1.5210,1.1452,2.3646,0.0,0.0,0.0,0.0,1.0,NaN,NaN,488771.0,9.2096,-133.830002,-8.2140,-8.9025,30.410400,-12.247200,3.6311,1.5714,NaN,NaN,NaN,92.084000,2213.0,6.479,529438.0,54.021900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,L,R,BAL,MIN,B,NaN,Bot,180331_231656,Infield shift,1.0,NaN,False,False,False,True,0,0,0,0,60.324272,27.808220,28.756958,37.196262,0.519561,0.133333,0.203333,29.661421,34.715027,26.459145,47.881355,0.470353,0.138158,0.111842,10.014306,38.297871,38.679245,24.637682,0.462878,0.156250,0.265625,0.000000,0,0,Kyle Gibson,NaN,ball,NaN,4-Seam Fastball,Standard,00,neutral,fastball
10019,FT,2018-03-31,15181,91.199997,-2.2699,6.3632,448801.0,502043.0,8.0,1.0,2018.0,-1.4111,1.0147,-0.2476,1.8754,0.0,0.0,0.0,0.0,1.0,130.669998,195.059998,488771.0,8.2048,-132.487595,-7.6040,-18.5795,29.222300,-18.640699,3.5538,1.5042,23.0,14.3,-21.0,91.592003,1961.0,6.692,529438.0,53.808201,0.15,0.138,0.0,1.0,0.0,0.0,2.0,5.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,L,R,BAL,MIN,X,ground_ball,Bot,180331_231716,Infield shift,1.0,NaN,False,False,False,False,0,1,1,0,60.324272,27.808220,28.756958,37.196262,0.519561,0.133333,0.203333,29.661421,34.715027,26.459145,47.881355,0.470353,0.138158,0.111842,10.014306,38.297871,38.679245,24.637682,0.462878,0.156250,0.265625,0.000000,1,0,Kyle Gibson,field_out,hit_into_play,"Chris Davis grounds out softly, pitcher Kyle G...",2-Seam Fastball,Standard,10,behind,fastball
10020,FF,2018-03-31,15167,92.900002,-1.8539,6.4785,592518.0,502043.0,9.0,NaN,2018.0,-0.6207,1.6913,0.6782,1.8998,0.0,0.0,0.0,1.0,1.0,NaN,NaN,488771.0,7.9316,-134.882996,-9.7386,-9.4737,30.538200,-9.245200,3.8157,1.8805,NaN,NaN,NaN,93.452003,2244.0,6.831,529438.0,53.669300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,BAL,MIN,S,NaN,Bot,180331_231759,Standard,1.0,NaN,False,False,False,False,0,0,1,0,60.031658,30.121817,43.618515,28.175896,0.439553,0.199495,0.176768,30.668777,38.163265,39.130436,28.421053,0.371922,0.163551,0.186916,9.259992,40.397350,40.740742,10.843373,0.425091,0.205479,0.315068,0.039573,0,0,Kyle Gibson,NaN,called_strike,NaN,4-Seam Fastball,Standard,00,neutral,fastball
10021,SL,2018-03-31,15153,85.199997,-2.1695,6.3682,592518.0,502043.0,14.0,NaN,2018.0,0.6245,0.2967,2.3338,1.7924,0.0,0.0,0.0,1.0,1.0,NaN,NaN,488771.0,9.2954,-123.751099,-4.9790,4.4722,25.264400,-28.358299,3.6814,1.7561,NaN,NaN,NaN,84.822998,2497.0,6.199,529438.0,54.301300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,BAL,MIN,B,NaN,Bot,180331_231814,Standard,1.0,NaN,Fa

In [0]:
#make list of pitchers (for filling in Nans in batting stat later for pitcher slot in batting order)

pitcher_list = df['pitcher'].unique().tolist()

#on rare occassions in extra-inning games, a position player is called in to pitch: to avoid categorizing
#him as a pitcher we'll only count pitchers with over 100 pitches:
for pitcher in pitcher_list:
    if len(df[df['pitcher'] == pitcher]) < 100:
        pitcher_list.remove(pitcher)

## Add the pitcher prior tendencies (Pitcher Scouting Report)

In [0]:
#Todo: consider feature engineering more features here (see blueprint notebook ideas)
#Add code to update the pitcher tendencies iteratively month by month to avoid leakage (see pre-processing step2)

In [291]:
%%time
#read in 2017 pre_procesed step 1 pickle and filter for pitcher_id
df17 = pd.read_pickle('/content/RC-v1.1--Data-Cleaning-and-Feature-Engineering/data/preprocessed_2017.pkl', compression = 'zip')
pitcher_df17 = df17[df17['pitcher'] == pitcher_id]

CPU times: user 2.07 s, sys: 84.9 ms, total: 2.15 s
Wall time: 2.16 s


In [0]:
#call the pitcher scouting report function and add to the df:

def get_pitch_tendencies(pitcher_df):
    #assign the normalized value counts for this pitchers pitch types to a dictionary
    pitcher_tendencies_overall = pitcher_df['pitch_type'].value_counts(normalize=True).to_dict()
    
    #initialize empty dict for count categories tendencies
    pitcher_tendencies_by_count = {}
    
    #loop over each count category and get the pitchers tendencies and add to the dict
    for cat in pitcher_df['count_cat'].unique().tolist():
        subset = pitcher_df[pitcher_df['count_cat'] == cat]
        pitcher_tendencies_by_count[cat] = subset['pitch_type'].value_counts(normalize=True).to_dict()
    return pitcher_tendencies_overall, pitcher_tendencies_by_count

def make_tendency_features(pitcher_df, pitcher_tendencies_overall, pitcher_tendencies_by_count):
    df = pitcher_df.copy()
    
    pitch_types = pitcher_tendencies_overall.keys()
    
    for pitch_type in pitch_types:
        overall_feature = 'overall_' + pitch_type + '_perc'
        count_cat_feature = 'count_cat_' + pitch_type + '_perc'
        
        def get_overall_perc(x):
            return pitcher_tendencies_overall[x]
        def get_by_count_perc(x):
            try:
                return pitcher_tendencies_by_count[x][pitch_type]
            except KeyError:
                return 0
        
        df[overall_feature] = pitch_type
        df[overall_feature] = df[overall_feature].apply(get_overall_perc)
        df[count_cat_feature] = df['count_cat'].apply(get_by_count_perc)
    return df

In [0]:
start_dates = ['2018-03-29', '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01', 
               '2018-09-01', '2019-03-28', '2019-05-01', '2019-06-01', '2019-07-01', 
               '2019-08-01']

end_dates =  ['2018-04-30', '2018-05-31', '2018-06-30', '2018-07-31', '2018-08-31', 
              '2018-10-01', '2019-04-30', '2019-05-31', '2019-06-30', '2019-07-31', 
              '2019-08-31']

  
def add_pitcher_scouting_report(pitcher_df, pitcher_df17, start_dates, end_dates):
    df = pd.concat([pitcher_df, pitcher_df17], sort=False)
    
    #initialize empty list to store dfs (concat them together later)
    df_list = []
    
    #iterate over each period
    for i in range(len(start_dates)):
      
        #make the prior and current dfs:
        prior_df = df[df['game_date'] < start_dates[i]]
        current_df = df[(df['game_date'] >= start_dates[i]) & (df['game_date'] <= end_dates[i])]
        
        #get the pitch tendencies from prior:
        pitcher_tendencies_overall, pitcher_tendencies_by_count = get_pitch_tendencies(prior_df)
        
        #make the pitch tendencies features on current:
        current_df = make_tendency_features(current_df, pitcher_tendencies_overall, pitcher_tendencies_by_count)
        
        #append the df to the list
        df_list.append(current_df)
    
    df = pd.concat(df_list, sort=False)
    return df

In [294]:
%%time
pitcher_df = add_pitcher_scouting_report(pitcher_df, pitcher_df17, start_dates, end_dates)

CPU times: user 646 ms, sys: 6.02 ms, total: 652 ms
Wall time: 656 ms


In [295]:
pitcher_df.isna().sum()

pitch_type                            0
game_date                             0
index                                 0
release_speed                         3
release_pos_x                         3
release_pos_z                         3
batter                                0
pitcher                               0
zone                                  3
hit_location                       4455
game_year                             0
pfx_x                                 3
pfx_z                                 3
plate_x                               3
plate_z                               3
on_3b                                 0
on_2b                                 0
on_1b                                 0
outs_when_up                          0
inning                                0
hc_x                               4740
hc_y                               4740
fielder_2                             1
vx0                                   3
vy0                                   3


## Add Game Features:

#### Batting Order, Game pitch_count, & Trailing Pitch Features

In [0]:
#Todo: add trailing 15 pitch features

In [0]:
def make_game_batting_order(game_df):
    game_df = game_df.sort_values(by=['at_bat_number', 'pitch_number'])
    all_batters = game_df['batter'].unique().tolist()
    #re-set the at_bat_number for the game to be sequential starting at 1
    at_bat_keys = game_df['at_bat_number'].unique().tolist()
    at_bat_values = range(1, len(at_bat_keys)+1)
    at_bat_map = dict(zip(at_bat_keys, at_bat_values))
    game_df['at_bat_number'] = game_df['at_bat_number'].replace(at_bat_map)
    
    #get the first 9 batter ids
    first_9_batter_subset = game_df[game_df['at_bat_number'] < 10]
    first_9_batters = first_9_batter_subset['batter'].unique().tolist()
    
    #map the batter id to batting order position 1-9
    batting_order_map = dict(zip(first_9_batters, range(1,10)))
    
    #for anyone else who bats later in the game, assign 'PH' (pinch hitter) to their batting order slot
    other_batters = list(set(all_batters) - set(first_9_batters))
    if len(other_batters) > 0:
        for batter in other_batters:
            batting_order_map[batter] = 'PH'
            
  
    try:
        game_df['batting_order_slot'] = game_df['batter'].apply(lambda x: batting_order_map[x])    
    except KeyError:
        game_df = None
        return game_df
    
    game_df['pitcher_AB'] = game_df['batter'].apply(lambda x: True if x in pitcher_list else False)
    game_df['batting_order_slot'] = game_df['batting_order_slot'].where(game_df['pitcher_AB'] == False, other='pitcher')
    return game_df
    
    
def make_game_pitchcount_and_trailing_pitch_features(pitcher_df, pitcher_list):
    df = pitcher_df.copy()
    
    print('#pitches in df before: ' + str(len(df)))
    
    pitcher_tendencies_overall, pitcher_tendencies_by_count = get_pitch_tendencies(df)
    games = df['game_pk'].unique().tolist()
    
    #take the first game and make the pitch count feature
    first_game_df = df[df['game_pk'] == games[0]].copy()
    first_game_df['pitch_count'] = range(1, first_game_df.shape[0] + 1)
    
    #make the L1_pitch type feature:
    first_game_df['L1_pitch_type'] = first_game_df['pitch_type'].shift(periods=1)
    first_game_df['L1_pitch_result'] = first_game_df['type'].shift(periods=1)
    first_game_df['L1_pitch_result'] = first_game_df['L1_pitch_result'].replace({np.nan:'first pitch'})
    first_game_df['L1_pitch_zone'] = first_game_df['zone'].shift(periods=1)
    first_game_df['L1_pitch_zone'] = first_game_df['L1_pitch_zone'].fillna(-1)
    
    #overall strike % (to fill in for first 5 pitches L5_strike_perc)
    overall_strike_perc = df['type'].value_counts(normalize=True)['S'] * 100
    
    #make the trailing 5 pitches:
    for index, row in first_game_df.iterrows():
        #fill NaNs for L1_pitch using same method as when pitch_type was missing
        if row['pitch_count'] == 1:
            random_pitch = random.choices(population=list(pitcher_tendencies_overall.keys()), 
                               weights=list(pitcher_tendencies_overall.values()), 
                               k=1)[0]
            first_game_df.at[index, 'L1_pitch_type'] = random_pitch
            
        #for the first 5 rows, use overall pitcher tendencies    
        if row['pitch_count'] < 6:
            #fill with overall tendencies
            for pitch in list(pitcher_tendencies_overall.keys()):
                feature = 'L5_' + pitch + '_perc'
                first_game_df.at[index, feature] = pitcher_tendencies_overall[pitch] * 100  
                #strike %
                first_game_df.at[index, 'L5_strike_perc'] = overall_strike_perc
        else:
            current_pitch = first_game_df.at[index, 'pitch_count']
            #make a subset of the prev 5 pitches
            subset = first_game_df[(first_game_df['pitch_count'] > current_pitch - 6) & (first_game_df['pitch_count'] < current_pitch)]
            #grab the value count percentages for the last 5 pitches
            subset_percentages = subset['pitch_type'].value_counts(normalize=True).to_dict()
            try:
                L5_strike_perc = subset['type'].value_counts(normalize=True)['S'] * 100
            except KeyError:
                L5_strike_perc = 0
                
            first_game_df.at[index, 'L5_strike_perc'] = L5_strike_perc
            
            #iterate over all possible pitch types this pitcher throws:
            for pitch in list(pitcher_tendencies_overall.keys()):
                feature = 'L5_' + pitch + '_perc'
                #if he has thrown that pitch type in last 5
                try:
                    first_game_df.at[index, feature] = subset_percentages[pitch] * 100
                #except for when he hasnt thrown that type in last 5
                except:
                    first_game_df.at[index, feature] = 0
                    
    #apply the battting order features to the game:                
    first_game_df = make_game_batting_order(first_game_df)
    
    #iterate the same process for the rest of his games:
    for game in games[1:]:
        game_df = df[df['game_pk'] == game].copy() #get df for that game only
        game_df['pitch_count'] = range(1, game_df.shape[0] + 1) #make the pitch count for the game
        game_df['L1_pitch_type'] = game_df['pitch_type'].shift(periods=1)
        game_df['L1_pitch_result'] = game_df['type'].shift(periods=1)
        game_df['L1_pitch_result'] = game_df['L1_pitch_result'].replace({np.nan:'first pitch'})
        game_df['L1_pitch_zone'] = game_df['zone'].shift(periods=1)
        game_df['L1_pitch_zone'] = game_df['L1_pitch_zone'].fillna(0)
        
        #make the trailing 5 pitches:
        for index, row in game_df.iterrows():
            #fill NaNs for L1_pitch using same method as when pitch_type was missing
            if row['pitch_count'] == 1:
                random_pitch = random.choices(population=list(pitcher_tendencies_overall.keys()), 
                               weights=list(pitcher_tendencies_overall.values()), 
                               k=1)[0]
                game_df.at[index, 'L1_pitch_type'] = random_pitch
            
            if row['pitch_count'] < 6:
                #fill with overall tendencies
                for pitch in list(pitcher_tendencies_overall.keys()):
                    feature = 'L5_' + pitch + '_perc'
                    game_df.at[index, feature] = pitcher_tendencies_overall[pitch] * 100
                    #strike %
                    game_df.at[index, 'L5_strike_perc'] = overall_strike_perc
            else:
                current_pitch = game_df.at[index, 'pitch_count']
                subset = game_df[(game_df['pitch_count'] > current_pitch - 6) & (game_df['pitch_count'] < current_pitch)]
                subset_percentages = subset['pitch_type'].value_counts(normalize=True).to_dict()
                
                try:
                    L5_strike_perc = subset['type'].value_counts(normalize=True)['S'] * 100
                except KeyError:
                    L5_strike_perc = 0
                game_df.at[index, 'L5_strike_perc'] = L5_strike_perc
                for pitch in list(pitcher_tendencies_overall.keys()):
                    feature = 'L5_' + pitch + '_perc'
                    try:
                        game_df.at[index, feature] = subset_percentages[pitch] * 100
                    except:
                        game_df.at[index, feature] = 0
                        
        #apply the battting order features to the game:                
        game_df = make_game_batting_order(game_df)
        if game_df.empty:
            print('skipping game because of bat data: ' + str(game))
            continue
        
        #concatenate that game w/ updated pitch count and trailing pitches w/ prev games
        if game_df['game_pk'].values[0] == games[1]:
            new_df = pd.concat([first_game_df, game_df]) #concat the game_df w/ the first game
        else:
            new_df = pd.concat([new_df, game_df]) #concat the game_df w/ the previous games
    
    print('# pitches in df after: ' + str(len(new_df)))
    
    return new_df

In [298]:
%%time
pitcher_df = make_game_pitchcount_and_trailing_pitch_features(pitcher_df, pitcher_list)

#pitches in df before: 5749
# pitches in df after: 5749
CPU times: user 30.8 s, sys: 8.87 ms, total: 30.8 s
Wall time: 30.8 s


In [299]:
pitcher_df.head()

,pitch_type,game_date,index,release_speed,release_pos_x,release_pos_z,batter,pitcher,zone,hit_location,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,fielder_2,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,stand,p_throws,home_team,away_team,type,bb_type,inning_topbot,sv_id,if_fielding_alignment,score_diff,on_1b_id,ball_high,ball_low,ball_left,ball_right,bases_loaded,batter_swung,in_strikezone,chased,fastball_perc_faced,fastball_chase_perc,fastball_bip_swung_perc,fastball_taken_strike_perc,fastball_est_woba,fastball_babip,fastball_iso_value,breaking_perc_faced,breaking_chase_perc,breaking_bip_swung_perc,breaking_taken_strike_perc,breaking_est_woba,breaking_babip,breaking_iso_value,offspeed_perc_faced,offspeed_chase_perc,offspeed_bip_swung_perc,offspeed_taken_strike_perc,offspeed_est_woba,offspeed_babip,offspeed_iso_value,pitchout_perc_faced,balls,strikes,player_name,events,description,des,pitch_name,of_fielding_alignment,_count,count_cat,pitch_cat,overall_FT_perc,count_cat_FT_perc,overall_FF_perc,count_cat_FF_perc,overall_SL_perc,count_cat_SL_perc,overall_CH_perc,count_cat_CH_perc,overall_CU_perc,count_cat_CU_perc,pitch_count,L1_pitch_type,L1_pitch_result,L1_pitch_zone,L5_FT_perc,L5_strike_perc,L5_FF_perc,L5_SL_perc,L5_CH_perc,L5_CU_perc,batting_order_slot,pitcher_AB
10018,FF,2018-03-31,15196,92.300003,-1.9879,6.6027,448801.0,502043.0,14.0,NaN,2018.0,-0.5666,1.5210,1.1452,2.3646,0.0,0.0,0.0,0.0,1.0,NaN,NaN,488771.0,9.2096,-133.830002,-8.2140,-8.9025,30.410400,-12.247200,3.6311,1.5714,NaN,NaN,NaN,92.084000,2213.0,6.479,529438.0,54.021900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,L,R,BAL,MIN,B,NaN,Bot,180331_231656,Infield shift,1.0,NaN,False,False,False,True,0,0,0,0,60.324272,27.808220,28.756958,37.196262,0.519561,0.133333,0.203333,29.661421,34.715027,26.459145,47.881355,0.470353,0.138158,0.111842,10.014306,38.297871,38.679245,24.637682,0.462878,0.156250,0.265625,0.000000,0,0,Kyle Gibson,NaN,ball,NaN,4-Seam Fastball,Standard,00,neutral,fastball,0.354023,0.380558,0.224521,0.254395,0.172414,0.135471,0.145594,0.113754,0.103448,0.115822,1,FF,first pitch,-1.0,33.518873,43.31188,21.13411,20.855801,12.906592,11.584623,1,False
10019,FT,2018-03-31,15181,91.199997,-2.2699,6.3632,448801.0,502043.0,8.0,1.0,2018.0,-1.4111,1.0147,-0.2476,1.8754,0.0,0.0,0.0,0.0,1.0,130.669998,195.059998,488771.0,8.2048,-132.487595,-7.6040,-18.5795,29.222300,-18.640699,3.5538,1.5042,23.0,14.3,-21.0,91.592003,1961.0,6.692,529438.0,53.808201,0.15,0.138,0.0,1.0,0.0,0.0,2.0,1.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,L,R,BAL,MIN,X,ground_ball,Bot,180331_231716,Infield shift,1.0,NaN,False,False,False,False,0,1,1,0,60.324272,27.808220,28.756958,37.196262,0.519561,0.133333,0.203333,29.661421,34.715027,26.459145,47.881355,0.470353,0.138158,0.111842,10.014306,38.297871,38.679245,24.637682,0.462878,0.156250,0.265625,0.000000,1,0,Kyle Gibson,field_out,hit_into_play,"Chris Davis grounds out softly, pitcher Kyle G...",2-Seam Fastball,Standard,10,behind,fastball,0.354023,0.444211,0.224521,0.265263,0.172414,0.088421,0.145594,0.160000,0.103448,0.042105,2,FF,B,14.0,33.518873,43.31188,21.13411,20.855801,12.906592,11.584623,1,False
10020,FF,2018-03-31,15167,92.900002,-1.8539,6.4785,592518.0,502043.0,9.0,NaN,2018.0,-0.6207,1.6913,0.6782,1.8998,0.0,0.0,0.0,1.0,1.0,NaN,NaN,488771.0,7.9316,-134.882996,-9.7386,-9.4737,30.538200,-9.245200,3.8157,1.8805,NaN,NaN,NaN,93.452003,2244.0,6.831,529438.0,53.669300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,BAL,MIN,S,NaN,Bot,180331_231759,Standard,1.0,NaN,False,False,False,False,0,0,1,0,60.031658,30.121

## Fill Batting Scouting Report NaNs

#### Load in the batting_order_slot map pickle

In [0]:
batting_order_slot_map = pickle.load(open("/content/RC-v1.1--Data-Cleaning-and-Feature-Engineering/data/batting_order_slot_map.pkl", "rb"))

In [301]:
batting_order_slot_map

{1: {'breaking_babip': 0.1872640385570563,
  'breaking_bip_swung_perc': 36.89605241827341,
  'breaking_chase_perc': 31.577836069125617,
  'breaking_est_woba': 0.3397664965875092,
  'breaking_iso_value': 0.13101442385072887,
  'breaking_perc_faced': 26.340695895944986,
  'breaking_taken_strike_perc': 35.5666746896674,
  'fastball_babip': 0.2154025865714933,
  'fastball_bip_swung_perc': 40.45598656570413,
  'fastball_chase_perc': 27.84080495082605,
  'fastball_est_woba': 0.37223512074559534,
  'fastball_iso_value': 0.15691701622442916,
  'fastball_perc_faced': 62.765084187795196,
  'fastball_taken_strike_perc': 31.962136660953938,
  'offspeed_babip': 0.20294449910457169,
  'offspeed_bip_swung_perc': 40.37310697509579,
  'offspeed_chase_perc': 36.54483903027876,
  'offspeed_est_woba': 0.32781539532817017,
  'offspeed_iso_value': 0.13571896819088838,
  'offspeed_perc_faced': 10.883704814573973,
  'offspeed_taken_strike_perc': 23.32780224123913,
  'pitchout_perc_faced': 0.010515101697300902

In [0]:
batter_cols = ['fastball_perc_faced','fastball_chase_perc','fastball_bip_swung_perc', 'fastball_taken_strike_perc',
               'fastball_est_woba', 'fastball_babip', 'fastball_iso_value', 'breaking_perc_faced', 'breaking_chase_perc',
               'breaking_bip_swung_perc', 'breaking_taken_strike_perc', 'breaking_est_woba', 'breaking_babip', 
               'breaking_iso_value', 'offspeed_perc_faced', 'offspeed_chase_perc', 'offspeed_bip_swung_perc',
               'offspeed_taken_strike_perc', 'offspeed_est_woba', 'offspeed_babip', 'offspeed_iso_value',
               'pitchout_perc_faced']

def fill_batting_nans(pitcher_df, batting_order_slot_map):
    df = pitcher_df.copy()
    for slot in df['batting_order_slot'].unique().tolist():
        subset = df[df['batting_order_slot'] == slot].copy()
        df = df.drop(subset.index)
        for col in batter_cols:
            subset[col] = subset[col].fillna(batting_order_slot_map[slot][col])
        df = pd.concat([df, subset])
        print('finished w/ slot: ' + str(slot))
    return df

In [303]:
%%time
pitcher_df = fill_batting_nans(pitcher_df, batting_order_slot_map)

finished w/ slot: 1
finished w/ slot: 2
finished w/ slot: 3
finished w/ slot: 4
finished w/ slot: 5
finished w/ slot: 6
finished w/ slot: 7
finished w/ slot: 8
finished w/ slot: 9
finished w/ slot: pitcher
finished w/ slot: PH
CPU times: user 386 ms, sys: 6.03 ms, total: 392 ms
Wall time: 393 ms


In [304]:
pitcher_df.isna().sum()

pitch_type                            0
game_date                             0
index                                 0
release_speed                         3
release_pos_x                         3
release_pos_z                         3
batter                                0
pitcher                               0
zone                                  3
hit_location                       4455
game_year                             0
pfx_x                                 3
pfx_z                                 3
plate_x                               3
plate_z                               3
on_3b                                 0
on_2b                                 0
on_1b                                 0
outs_when_up                          0
inning                                0
hc_x                               4740
hc_y                               4740
fielder_2                             1
vx0                                   3
vy0                                   3


In [305]:
pitcher_df.sort_values(by=['game_date', 'game_pk', 'at_bat_number']).head(10)

,pitch_type,game_date,index,release_speed,release_pos_x,release_pos_z,batter,pitcher,zone,hit_location,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,fielder_2,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,stand,p_throws,home_team,away_team,type,bb_type,inning_topbot,sv_id,if_fielding_alignment,score_diff,on_1b_id,ball_high,ball_low,ball_left,ball_right,bases_loaded,batter_swung,in_strikezone,chased,fastball_perc_faced,fastball_chase_perc,fastball_bip_swung_perc,fastball_taken_strike_perc,fastball_est_woba,fastball_babip,fastball_iso_value,breaking_perc_faced,breaking_chase_perc,breaking_bip_swung_perc,breaking_taken_strike_perc,breaking_est_woba,breaking_babip,breaking_iso_value,offspeed_perc_faced,offspeed_chase_perc,offspeed_bip_swung_perc,offspeed_taken_strike_perc,offspeed_est_woba,offspeed_babip,offspeed_iso_value,pitchout_perc_faced,balls,strikes,player_name,events,description,des,pitch_name,of_fielding_alignment,_count,count_cat,pitch_cat,overall_FT_perc,count_cat_FT_perc,overall_FF_perc,count_cat_FF_perc,overall_SL_perc,count_cat_SL_perc,overall_CH_perc,count_cat_CH_perc,overall_CU_perc,count_cat_CU_perc,pitch_count,L1_pitch_type,L1_pitch_result,L1_pitch_zone,L5_FT_perc,L5_strike_perc,L5_FF_perc,L5_SL_perc,L5_CH_perc,L5_CU_perc,batting_order_slot,pitcher_AB
10018,FF,2018-03-31,15196,92.300003,-1.9879,6.6027,448801.0,502043.0,14.0,NaN,2018.0,-0.5666,1.5210,1.1452,2.3646,0.0,0.0,0.0,0.0,1.0,NaN,NaN,488771.0,9.2096,-133.830002,-8.2140,-8.902500,30.410400,-12.247200,3.6311,1.5714,NaN,NaN,NaN,92.084000,2213.0,6.479,529438.0,54.021900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,L,R,BAL,MIN,B,NaN,Bot,180331_231656,Infield shift,1.0,NaN,False,False,False,True,0,0,0,0,60.324272,27.808220,28.756958,37.196262,0.519561,0.133333,0.203333,29.661421,34.715027,26.459145,47.881355,0.470353,0.138158,0.111842,10.014306,38.297871,38.679245,24.637682,0.462878,0.156250,0.265625,0.000000,0,0,Kyle Gibson,NaN,ball,NaN,4-Seam Fastball,Standard,00,neutral,fastball,0.354023,0.380558,0.224521,0.254395,0.172414,0.135471,0.145594,0.113754,0.103448,0.115822,1,FF,first pitch,-1.0,33.518873,43.31188,21.13411,20.855801,12.906592,11.584623,1,False
10019,FT,2018-03-31,15181,91.199997,-2.2699,6.3632,448801.0,502043.0,8.0,1.0,2018.0,-1.4111,1.0147,-0.2476,1.8754,0.0,0.0,0.0,0.0,1.0,130.669998,195.059998,488771.0,8.2048,-132.487595,-7.6040,-18.579500,29.222300,-18.640699,3.5538,1.5042,23.0,14.300000,-21.0,91.592003,1961.0,6.692,529438.0,53.808201,0.15,0.138,0.0,1.0,0.0,0.0,2.0,1.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,L,R,BAL,MIN,X,ground_ball,Bot,180331_231716,Infield shift,1.0,NaN,False,False,False,False,0,1,1,0,60.324272,27.808220,28.756958,37.196262,0.519561,0.133333,0.203333,29.661421,34.715027,26.459145,47.881355,0.470353,0.138158,0.111842,10.014306,38.297871,38.679245,24.637682,0.462878,0.156250,0.265625,0.000000,1,0,Kyle Gibson,field_out,hit_into_play,"Chris Davis grounds out softly, pitcher Kyle G...",2-Seam Fastball,Standard,10,behind,fastball,0.354023,0.444211,0.224521,0.265263,0.172414,0.088421,0.145594,0.160000,0.103448,0.042105,2,FF,B,14.0,33.518873,43.31188,21.13411,20.855801,12.906592,11.584623,1,False
10020,FF,2018-03-31,15167,92.900002,-1.8539,6.4785,592518.0,502043.0,9.0,NaN,2018.0,-0.6207,1.6913,0.6782,1.8998,0.0,0.0,0.0,1.0,1.0,NaN,NaN,488771.0,7.9316,-134.882996,-9.7386,-9.473700,30.538200,-9.245200,3.8157,1.8805,NaN,NaN,NaN,93.452003,2244.0,6.831,529438.0,53.669300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,BAL,MIN,S,NaN,Bot,180331_231759,Standard,1.0,NaN,False,False,False,False,0,0,1,0,60.03

## Add Pitcher / Batter Matchup History Report

In [0]:
##Todo

## Choose features to use in model input vector, re-cast datatypes, encode categorical variables (onehot and binary), scale appropriately (Standard vs MinMax vs Robust), and check for any remaining NaNs and choose imputation strategy

#### Train/test Split (IMO use April/2017 - end of June/2019 as Train, and June/July/August 2019 as Test)

In [0]:
#Todo